In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [28]:
df = pd.read_csv("train.csv")

C:\Users\Asus\AppData\Local\Temp\ipykernel_11212\2292805398.py:1: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")


In [29]:
df = df.drop(columns=['surname','forename','dob','positionText_x','number','time_x','fastestLap','rank','fastestLapTime','max_speed',])
df = df.drop(columns=['grand_prix','date','time_y','url_x','fp1_date',
                      'fp2_date','fp3_date','quali_date','quali_time','sprint_date','sprint_time','driverRef',
                     'driver_num','driver_code','nationality','url_y','positionText_y','constructorRef',
                     'company','nationality_y','url'])
df = df.drop(columns=['fp1_time','fp2_time','fp3_time'])
df = df.drop(columns=['positionOrder'])
df = df.drop(columns=['resultId',])
df = df.drop(columns=['driverStandingsId','position_x'])
df = df.drop(columns=['statusId'])
df['is_finished'] = df['status'].apply(lambda x: 1 if x == 'Finished' else 0)
df['avg_grid_position'] = df.groupby(['driverId','year'])['grid'].transform(lambda x: x.expanding().mean().shift(1))

# Cumulative points up to the current lap, excluding the current lap's points
df['cumulative_points'] = df.groupby(['driverId', 'racerId'])['points_y'].cumsum().shift(1)


In [30]:
df=df.drop(columns=['status'])

In [31]:
print(df.columns)

Index(['racerId', 'driverId', 'constructorId', 'grid', 'points', 'laps',
       'timetaken_in_millisec', 'year', 'round', 'circuitId', 'raceId_y',
       'points_y', 'position', 'wins', 'result_driver_standing', 'is_finished',
       'avg_grid_position', 'cumulative_points'],
      dtype='object')


In [32]:
df=df.drop(columns=["timetaken_in_millisec"])

In [33]:
#take the correlation of the columns with position
print(df.corr()['position'])

racerId                   0.146104
driverId                  0.241365
constructorId             0.165706
grid                      0.235240
points                   -0.220597
laps                     -0.084758
year                     -0.270743
round                    -0.068691
circuitId                 0.093777
raceId_y                  0.100520
points_y                 -0.421678
position                  1.000000
wins                     -0.388797
result_driver_standing    0.092763
is_finished              -0.235115
avg_grid_position         0.278260
cumulative_points        -0.233852
Name: position, dtype: float64


In [34]:
df_test=pd.read_csv("validation.csv")
#df and df_test have the intersection of columns in common
cols=list(set(df.columns).intersection(set(df_test.columns)))
df=df[cols]
df_test=df_test[cols]



C:\Users\Asus\AppData\Local\Temp\ipykernel_11212\2719348433.py:1: DtypeWarning: Columns (13,16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test=pd.read_csv("validation.csv")


In [35]:
#correlation of the columns with position
print(df_test.corr()['position'])

racerId                  -0.025206
grid                      0.365640
year                     -0.022140
laps                     -0.070981
result_driver_standing    0.085667
position                  1.000000
round                    -0.004298
circuitId                -0.006027
points                   -0.349921
points_y                 -0.641362
wins                     -0.504762
driverId                  0.320787
raceId_y                  0.113942
constructorId             0.081673
Name: position, dtype: float64


In [36]:
X=df.drop(columns=['position'])
y=df['position']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_test=df_test.drop(columns=['position'])
y_test=df_test['position']

In [37]:
y_train = np.where(y_train > 20, 20, y_train)
y_val = np.where(y_val > 20, 20, y_val)


In [38]:
y_train -= 1
y_val -= 1
y_test -= 1

In [39]:
y_test = np.where(y_test > 20, 20, y_test)  

In [14]:
!pip install lightgbm


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\Asus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [40]:
import lightgbm as lgb  

In [41]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

In [51]:

# Initialize the model after the final split, ensuring it's aware of all classes
params = {
    'objective': 'regression',  # For regression tasks
    'metric': 'rmse',           # Use RMSE as the evaluation metric
    'boosting_type': 'gbdt',    # Traditional Gradient Boosting Decision Tree
    'num_leaves': 31,           # Number of leaves in trees
    'learning_rate': 0.05,      # Learning rate
    'feature_fraction': 0.9,    # Fraction of features to be used in each iteration
    'bagging_fraction': 0.8,    # Fraction of data to be used for each iteration
    'bagging_freq': 5,          # Perform bagging every 5 iterations
    'verbose': 0,               # Don't print messages during training
    'num_iterations': 20000,      # Number of boosting iterations
    'early_stopping_rounds': 10 # Stop training if the validation score doesn't improve for 10 rounds
}


model = lgb.train(params,
                  train_data,
                  valid_sets=[train_data, val_data],
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])  # Print progress


# Predict on the test set
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_rounded = np.round(y_pred).astype(int)
y_pred_clipped = np.clip(y_pred_rounded, 1, 20)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_clipped))

print("Predictions:")
print(y_pred_clipped)
print(f'RMSE: {rmse}')


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 6.83897	valid_1's rmse: 6.88001
[20]	training's rmse: 5.49415	valid_1's rmse: 5.53027
[30]	training's rmse: 4.88407	valid_1's rmse: 4.91573
[40]	training's rmse: 4.5914	valid_1's rmse: 4.61936
[50]	training's rmse: 4.43421	valid_1's rmse: 4.45983
[60]	training's rmse: 4.33229	valid_1's rmse: 4.35581
[70]	training's rmse: 4.25692	valid_1's rmse: 4.28007
[80]	training's rmse: 4.19649	valid_1's rmse: 4.21926
[90]	training's rmse: 4.13902	valid_1's rmse: 4.16189
[100]	training's rmse: 4.0893	valid_1's rmse: 4.11246
[110]	training's rmse: 4.04856	valid_1's rmse: 4.07225
[120]	training's rmse: 4.00722	valid_1's rmse: 4.03156
[130]	training's rmse: 3.97178	valid_1's rmse: 3.99651
[140]	training's rmse: 3.93747	valid_1's rmse: 3.9625
[150]	training's rmse: 3.90238	valid_1's rmse: 3.92843
[160]	training's rmse: 3.8718	valid_1's rmse: 3.89802
[170]	training's rmse: 3.84806	valid_1's rmse: 3.87409
[180]	training's 

In [52]:
X=pd.concat([X,X_test])
y_test_series = pd.Series(y_test)

# Now, concatenate y and y_test_series
y = pd.concat([y, y_test_series])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
# Continue with fitting the model as before
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

model = lgb.train(params,
                    train_data,
                    valid_sets=[train_data, val_data],
                    callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])  # Print progress

# Predict on the test set

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 6.65996	valid_1's rmse: 6.6786
[20]	training's rmse: 5.32921	valid_1's rmse: 5.34415
[30]	training's rmse: 4.72894	valid_1's rmse: 4.73932
[40]	training's rmse: 4.44301	valid_1's rmse: 4.45032
[50]	training's rmse: 4.29563	valid_1's rmse: 4.30076
[60]	training's rmse: 4.20042	valid_1's rmse: 4.20346
[70]	training's rmse: 4.12588	valid_1's rmse: 4.12838
[80]	training's rmse: 4.0702	valid_1's rmse: 4.07172
[90]	training's rmse: 4.01163	valid_1's rmse: 4.01381
[100]	training's rmse: 3.96205	valid_1's rmse: 3.96426
[110]	training's rmse: 3.91967	valid_1's rmse: 3.92201
[120]	training's rmse: 3.88409	valid_1's rmse: 3.88652
[130]	training's rmse: 3.85295	valid_1's rmse: 3.85499
[140]	training's rmse: 3.82419	valid_1's rmse: 3.8264
[150]	training's rmse: 3.79643	valid_1's rmse: 3.79927
[160]	training's rmse: 3.7738	valid_1's rmse: 3.7773
[170]	training's rmse: 3.74755	valid_1's rmse: 3.75186
[180]	training's r

In [53]:
pd_final = pd.read_csv("test.csv")

C:\Users\Asus\AppData\Local\Temp\ipykernel_11212\3429583794.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_final = pd.read_csv("test.csv")


In [54]:
pd_final = pd_final[X.columns]

# Predict the class for each row in pd_final
predictions = model.predict(pd_final)

# Increment the predictions by 1 to shift the range from 0-19 to 1-20
adjusted_predictions = predictions + 1

# Print the adjusted predictions
print(adjusted_predictions)

[ 4.61261799  4.41565168  6.12657317 ... 10.88282332  9.87722078
 10.63823114]


In [55]:
print(adjusted_predictions.shape)

(352928,)


In [56]:
result_driver_standing_array = pd_final['result_driver_standing'].values
print(result_driver_standing_array)

[ 298739826  299218806  299697786 ... 1881902325 1882424025 1882971810]


In [57]:

# Create a DataFrame from the two arrays
df_sub = pd.DataFrame({
    'predictions': adjusted_predictions,
    'result_driver_standing': result_driver_standing_array
})

# Display the DataFrame
print(df_sub)


        predictions  result_driver_standing
0          4.612618               298739826
1          4.415652               299218806
2          6.126573               299697786
3          4.903401               300176766
4          4.903401               300655746
...             ...                     ...
352923    16.285059              1880337225
352924    16.285059              1881380625
352925    10.882823              1881902325
352926     9.877221              1882424025
352927    10.638231              1882971810

[352928 rows x 2 columns]


In [58]:
df_sub.to_csv('submission.csv')